# Main

> Main application where the translation web service endpoints are defined.

In [ ]:
#| default_exp main

In [ ]:
#| hide
from nbdev.showdoc import *

# Imports

We are using `Flask` to define the web service endpoints, and we are using Hugging Face's `transformers` library to load the model and perform the translation.

In [ ]:
#| exports

from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Loading Models

We are loading the models from the `models` folder. The models are [`MarianMT`](https://huggingface.co/docs/transformers/model_doc/marian) models.

In [ ]:
#| export

def get_model(model_path):
    """Load a Hugging Face model and tokenizer from the specified directory"""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    return model, tokenizer

In [ ]:
#| exports
#| eval: false

# Load the models and tokenizers for each supported language
en_fr_model, en_fr_tokenizer = get_model('models/en_fr/')
fr_en_model, fr_en_tokenizer = get_model('models/fr_en/')

# Web Service Endpoints

The next step is to use Flask to create the English/French and French/English translation web service endpoints.

We first check if the path to the translation web service exists. The two paths currently defined are:

 - `/translate/en/fr`
 - `/translate/fr/en`

Once the container is running, we can test the web service endpoints using `curl`:

```bash
curl http://localhost:6000/translate/en/fr/ POST -H "Content-Type: application/json" -d '{"en_text":"Hello World!"}'
```

The output should be:

```json
{
  "fr_text": "Bonjour le monde!"
}
```

```bash
curl http://localhost:6000/translate/fr/en/ POST -H "Content-Type: application/json" -d '{"fr_text":"Bonjour le monde!"}'
```

The output should be:

```json
{
  "en_text": "Hello world!"
}
```

In [ ]:
#| export

app = Flask(__name__)

def is_translation_supported(from_lang, to_lang):
    """Check if the specified translation is supported"""
    supported_translations = ['en_fr', 'fr_en']
    return f'{from_lang}_{to_lang}' in supported_translations

@app.route('/translate/<from_lang>/<to_lang>/', methods=['POST'])
def translate_endpoint(from_lang, to_lang):
    """Translate text from one language to another. This function is 
    called when a POST request is sent to `/translate/<from_lang>/<to_lang>/`"""
    if not is_translation_supported(from_lang, to_lang):
        return jsonify({'error': 'Translation not supported'}), 400

    data = request.get_json()
    from_text = data.get(f'{from_lang}_text', '')

    if from_text:
        model = None
        tokenizer = None

        match from_lang:
            case 'en':        
                model = en_fr_model
                tokenizer = en_fr_tokenizer
            case 'fr':
                model = fr_en_model
                tokenizer = fr_en_tokenizer

        to_text = tokenizer.decode(model.generate(tokenizer.encode(from_text, return_tensors='pt')).squeeze(), skip_special_tokens=True)

        return jsonify({f'{to_lang}_text': to_text})
    else:
        return jsonify({'error': 'Text to translate not provided'}), 400

# Entrypoint

Finally, we define the entry point of the application. This is the file that will be executed when the container is run. It will run the Flask application on port 6000 and enables the debug mode.

In [ ]:
#| exports
#| eval: false

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=6000, debug=True)